In [1]:
using Flows
using ToySystems
using ToySystems.NineModeSystemEq
using Optim
include("9msctrl.jl")
include("Initial condition variation.jl")
include("Perturbation time.jl")
include("Contour Plots.jl")
include("Magnitude Rand.jl")
include("Objective function Monitor.jl")
include("AdjointControl.jl")
import LinearAlgebra: norm
using PyPlot; pygui(true)
using PyCall
np = pyimport("numpy")
sc = pyimport("scipy.optimize")
plt = pyimport("matplotlib.pyplot")
using DelimitedFiles
using CurveFit;

┌ Info: Recompiling stale cache file C:\Users\User\.julia\compiled\v1.2\ToySystems\chsON.ji for ToySystems [53c88820-6eff-5a8c-9a74-badd815e7be1]
└ @ Base loading.jl:1240
┌ Warning: Package ToySystems does not have Flows in its dependencies:
│ - If you have ToySystems checked out for development and have
│   added Flows as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ToySystems
└ Loading Flows into ToySystems from project dependency, future warnings for ToySystems are suppressed.


In [2]:
function indexpoint(y_val::AbstractVector, number ,FIRST = true)
    n = 1
    for i in 1:length(y_val)
        if y_val[i] == number
            n = i
            if FIRST == true
                break
            else
                n = n
            end
        else
            n = n
        end
    end
    return n
end

function indexpointrange(y_range, low, up)
    out = []
    for i in 1:length(y_range)
        if y_range[i]>=low && y_range[i]<=up
            out = push!(out, i)
        else
            dummy = 1
        end
    end
    return extrema(out)[1],extrema(out)[2]
end

y_val = readdlm("DecayProb_Corrected.txt")[1:201] #alpha = 2.5,3.0,3.5,4.0, Re = 350,400,450;
# 
# x_points = x_full[indexpoint(y_val, 1.0, false):indexpoint(y_val, extrema(y_val)[1], true)]
# y_points = y_val[indexpoint(y_val, 1.0, false):indexpoint(y_val, extrema(y_val)[1], true)]
# y1_points = log.(y_points)

# fit = curve_fit(LinearFit, x_points, y1_points)
# y0b = fit.(x_points)

# fig, ax = plt.subplots()
# ax.scatter(x_points,y1_points)
# ax.plot(x_points, y0b)

In [3]:
t1 = 1
t2 = 201
m = []
Re = 350
for j in 1:3
    fig, ax = plt.subplots()
    n = 2.5
    for i in 1:8
        y_val = readdlm("DecayProb_Corrected_2.txt")[t1:t2]
        x_full = 0.0:100.0:20000.0
        i1, i2 = indexpointrange(y_val, 0.010, 0.999)
        x_points = x_full[i1:i2]
        y_points = y_val[i1:i2]
        fit = curve_fit(ExpFit, x_points, y_points)
        y1_points = fit.(x_points)
        y2_points = log.(y1_points)
        m = push!(m, (y2_points[2]-y2_points[1])/100.0)
        ax.plot(x_full, y_val, color = "black", linewidth = 0.1)
        ax.plot(x_points, y1_points, linewidth = 1.5, label = "α = $n @ Re = $Re")
        n += 0.5
        t1 += 201
        t2 += 201
    end
    ax.legend(loc = 1)
    ylabel(L"P(t)")
    xlabel(L"t")
    Re += 50
end

In [4]:
# using LsqFit
# @. model(x, p) = p[1]*exp(-x*p[2]) + p[3]

# t1 = 1
# t2 = 201
# slope_val = []
# for j in 1:3
#     for i in 1:4
#         y_val = readdlm("DecayProb_Corrected.txt")[t1:t2]
#         x_full = 0.0:100.0:20000.0
#         i1, i2 = indexpointrange(y_val, 0.010, 0.999)
#         println(i1,i2)
#         x_points = x_full[i1:i2]
#         y_points = y_val[i1:i2]
#         p0 = [4, 0.01, 1.0]
#         fit = LsqFit.curve_fit(model, x_points, y_points, p0)
#         slope_val = push!(slope_val,coef(fit)[2])
#         t1 += 201
#         t2 += 201
#     end
# end
# slope_val

In [5]:
Re_350_alpha_2560 = readdlm("DecayProb_Corrected_2.txt")[1:1608]
Re_400_alpha_2560 = readdlm("DecayProb_Corrected_2.txt")[1609:3216]
Re_450_alpha_2560 = readdlm("DecayProb_Corrected_2.txt")[3217:4824];

In [6]:
fig,ax = plt.subplots()
Re = 350
t1=1
t2=201
n = 2.5
for i in 1:8
    ax.plot(0:100:20000, Re_350_alpha_2560[t1:t2],label = "α = $n @ Re = $Re")
    n+=0.5
    t1 = t2+1
    t2 = t2+201
end
ax.legend(loc = 1)
ylabel(L"P(t)")
xlabel(L"t")

fig,ax1 = plt.subplots()
Re = 400
t1=1
t2=201
n = 2.5
for i in 1:8
    ax1.plot(0:100:20000, Re_400_alpha_2560[t1:t2], label = "α = $n @ Re = $Re")
    n+=0.5
    t1 = t2+1
    t2 = t2+201
end
ax1.legend(loc = 1)
ylabel(L"P(t)")
xlabel(L"t")


fig,ax2 = plt.subplots()
Re = 450
t1=1
t2=201
n = 2.5
for i in 1:8
    ax2.plot(0:100:20000, Re_450_alpha_2560[t1:t2], label = "α = $n @ Re = $Re")
    n+=0.5
    t1 = t2+1
    t2 = t2+201
end
ax2.legend(loc = 1)
ylabel(L"P(t)")
xlabel(L"t")

PyObject Text(0.5, 23.082222222222214, '$t$')

In [7]:
Re_360_alpha_2560 = readdlm("DecayProb_Corrected_360_440_2.5.txt")[1:1608]
Re_370_alpha_2560 = readdlm("DecayProb_Corrected_360_440_2.5.txt")[1609:3216]
Re_380_alpha_2560 = readdlm("DecayProb_Corrected_360_440_2.5.txt")[3217:4824]
Re_390_alpha_2560 = readdlm("DecayProb_Corrected_360_440_2.5.txt")[4825:6432]
Re_410_alpha_2560 = readdlm("DecayProb_Corrected_360_440_2.5.txt")[6433:8040]
Re_420_alpha_2560 = readdlm("DecayProb_Corrected_360_440_2.5.txt")[8041:9648]
Re_430_alpha_2560 = readdlm("DecayProb_Corrected_360_440_2.5.txt")[9649:11256]
Re_440_alpha_2560 = readdlm("DecayProb_Corrected_360_440_2.5.txt")[11257:12864];

In [8]:
A1 = readdlm("DecayProb_275325375.txt")
A2 = readdlm("DecayProb_425475.txt")
A3 = readdlm("DecayProb_Corrected_525575.txt");

In [9]:
Re1 = reshape(Re_350_alpha_2560, 201, 8)
Re2 = reshape(Re_360_alpha_2560, 201, 8)
Re3 = reshape(Re_370_alpha_2560, 201, 8)
Re4 = reshape(Re_380_alpha_2560, 201, 8)
Re5 = reshape(Re_390_alpha_2560, 201, 8)
Re6 = reshape(Re_400_alpha_2560, 201, 8)
Re7 = reshape(Re_410_alpha_2560, 201, 8)
Re8 = reshape(Re_420_alpha_2560, 201, 8)
Re9 = reshape(Re_430_alpha_2560, 201, 8)
Re10 = reshape(Re_440_alpha_2560, 201, 8)
Re11 = reshape(Re_450_alpha_2560, 201, 8);

In [10]:
Re1int = hcat(reshape(A1[1:603], 201,3), reshape(A2[1+402:402+402], 201, 2))
Re2int = hcat(reshape(A1[604:1206], 201,3), reshape(A2[403+402:804+402], 201, 2))
Re3int = hcat(reshape(A1[1207:1809], 201,3), reshape(A2[805+402:1206+402], 201, 2))
Re4int = hcat(reshape(A1[1810:2412], 201,3), reshape(A2[1207+402:1608+402], 201, 2))
Re5int = hcat(reshape(A1[2413:3015], 201,3), reshape(A2[1609+402:2010+402], 201, 2))
Re6int = hcat(reshape(A1[3016:3618], 201,3), reshape(A2[2011+402:2412+402], 201, 2))
Re7int = hcat(reshape(A1[3619:4221], 201,3), reshape(A2[2413+402:2814+402], 201, 2))
Re8int = hcat(reshape(A1[4222:4824], 201,3), reshape(A2[2815+402:3216+402], 201, 2))
Re9int = hcat(reshape(A1[4825:5427], 201,3), reshape(A2[3217+402:3618+402], 201, 2))
Re10int = hcat(reshape(A1[5428:6030], 201,3), reshape(A2[3619+402:4020+402], 201, 2))
Re11int = hcat(reshape(A1[6031:6633], 201,3), reshape(A2[4021+402:4422+402], 201, 2));

In [11]:
Re1int = hcat(Re1int, reshape(A3[1:402], 201, 2))
Re2int = hcat(Re2int, reshape(A3[403:804], 201, 2))
Re3int = hcat(Re3int, reshape(A3[805:1206], 201, 2))
Re4int = hcat(Re4int, reshape(A3[1207:1608], 201, 2))
Re5int = hcat(Re5int, reshape(A3[1609:2010], 201, 2))
Re6int = hcat(Re6int, reshape(A3[2011:2412], 201, 2))
Re7int = hcat(Re7int, reshape(A3[2413:2814], 201, 2))
Re8int = hcat(Re8int, reshape(A3[2815:3216], 201, 2))
Re9int = hcat(Re9int, reshape(A3[3217:3618], 201, 2))
Re10int = hcat(Re10int, reshape(A3[3619:4020], 201, 2))
Re11int = hcat(Re11int, reshape(A3[4021:4422], 201, 2))

201×7 Array{Float64,2}:
 1.0    1.0    1.0    1.0  1.0  1.0    1.0  
 1.0    1.0    1.0    1.0  1.0  1.0    1.0  
 1.0    1.0    1.0    1.0  1.0  1.0    1.0  
 1.0    1.0    1.0    1.0  1.0  1.0    1.0  
 1.0    1.0    1.0    1.0  1.0  1.0    1.0  
 1.0    1.0    1.0    1.0  1.0  1.0    1.0  
 1.0    1.0    1.0    1.0  1.0  1.0    1.0  
 1.0    1.0    1.0    1.0  1.0  1.0    1.0  
 1.0    1.0    1.0    1.0  1.0  1.0    1.0  
 1.0    1.0    1.0    1.0  1.0  1.0    1.0  
 1.0    1.0    1.0    1.0  1.0  1.0    1.0  
 1.0    1.0    1.0    1.0  1.0  1.0    1.0  
 0.998  0.998  0.998  1.0  1.0  1.0    1.0  
 ⋮                              ⋮           
 0.0    0.0    0.0    0.0  0.0  0.224  0.302
 0.0    0.0    0.0    0.0  0.0  0.224  0.298
 0.0    0.0    0.0    0.0  0.0  0.222  0.294
 0.0    0.0    0.0    0.0  0.0  0.222  0.292
 0.0    0.0    0.0    0.0  0.0  0.222  0.29 
 0.0    0.0    0.0    0.0  0.0  0.222  0.288
 0.0    0.0    0.0    0.0  0.0  0.218  0.286
 0.0    0.0    0.0    0.0  0.0 

In [12]:
Re1_final = []
for i in 1:7
    Re1_final = push!(Re1_final, Re1[:,i])
    Re1_final = push!(Re1_final, Re1int[:,i])
end
Re1_final = push!(Re1_final, Re1[:,end])

Re2_final = []
for i in 1:7
    Re2_final = push!(Re2_final, Re2[:,i])
    Re2_final = push!(Re2_final, Re2int[:,i])
end
Re2_final = push!(Re2_final, Re2[:,end]);

Re3_final = []
for i in 1:7
    Re3_final = push!(Re3_final, Re3[:,i])
    Re3_final = push!(Re3_final, Re3int[:,i])
end
Re3_final = push!(Re3_final, Re3[:,end]);

Re4_final = []
for i in 1:7
    Re4_final = push!(Re4_final, Re4[:,i])
    Re4_final = push!(Re4_final, Re4int[:,i])
end
Re4_final = push!(Re4_final, Re4[:,end]);

Re5_final = []
for i in 1:7
    Re5_final = push!(Re5_final, Re5[:,i])
    Re5_final = push!(Re5_final, Re5int[:,i])
end
Re5_final = push!(Re5_final, Re5[:,end]);

Re6_final = []
for i in 1:7
    Re6_final = push!(Re6_final, Re6[:,i])
    Re6_final = push!(Re6_final, Re6int[:,i])
end
Re6_final = push!(Re6_final, Re6[:,end]);

Re7_final = []
for i in 1:7
    Re7_final = push!(Re7_final, Re7[:,i])
    Re7_final = push!(Re7_final, Re7int[:,i])
end
Re7_final = push!(Re7_final, Re7[:,end]);

Re8_final = []
for i in 1:7
    Re8_final = push!(Re8_final, Re8[:,i])
    Re8_final = push!(Re8_final, Re8int[:,i])
end
Re8_final = push!(Re8_final, Re8[:,end]);

Re9_final = []
for i in 1:7
    Re9_final = push!(Re9_final, Re9[:,i])
    Re9_final = push!(Re9_final, Re9int[:,i])
end
Re9_final = push!(Re9_final, Re9[:,end]);

Re10_final = []
for i in 1:7
    Re10_final = push!(Re10_final, Re10[:,i])
    Re10_final = push!(Re10_final, Re10int[:,i])
end
Re10_final = push!(Re10_final, Re10[:,end]);

Re11_final = []
for i in 1:7
    Re11_final = push!(Re11_final, Re11[:,i])
    Re11_final = push!(Re11_final, Re11int[:,i])
end
Re11_final = push!(Re11_final, Re11[:,end])

15-element Array{Any,1}:
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]                   
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]                   
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]                   
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]                   
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]                   
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]                   
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]                   
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]   

In [22]:
Re460570 = readdlm("DecayProb_460600.txt")
Re_ar = []
d1 = 1
d2 = 201
for i in 1:12
    Re_ar1 = []
    for j in 1:15
        Re_ar1 = push!(Re_ar1, Re460570[d1:d2])
        d1 = d2+1
        d2 = d2+201
    end
    Re_ar = push!(Re_ar,Re_ar1)
end

# for i in 1:12
#     fig,ax = plt.subplots()
#     n = 2.5
#     m = 460
#     for j in 1:15
#         ax.plot(0:100:20000,Re_ar[i][j], label = "$n @ Re=$m")
#         n+=0.25
#     end
#     ax.legend()
# end

In [23]:
Re_total = []
Re_total = push!(Re_total,Re1_final,Re2_final,Re3_final,Re4_final,Re5_final,Re6_final,Re7_final,Re8_final,Re9_final,Re10_final,Re11_final)
for i in 1:12
    Re_total = push!(Re_total, Re_ar[i])
end
Re_total
# Reyy = 350
# for i in 1:length(Re_total)
#     fig, ax = plt.subplots()
#     n1 = 2.5
#     for j in 1:15
#         plot(0:100:20000,Re_total[i][j], label = "Re = $Reyy α = $n1")
#         n1 += 0.25
#     end
#     Reyy+=10
#     ax.legend()
# end

23-element Array{Any,1}:
 Any[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.998  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.998  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 1.0, 1.0, 

In [37]:
x_full = 0.0:100.0:20000.0
Re_rangeu = 350:10:570
α_rangeu = 2.5:0.25:6.0
m_mat = zeros(length(α_rangeu), length(Re_rangeu))
for i in 1:length(Re_rangeu)
    for j in 1:length(α_rangeu)
        parameter = Re_total[i][j]
        i1, i2 = indexpointrange(parameter, 0.05, 0.999)
        x_points = x_full[i1:i2]
        y_points = parameter[i1:i2]
        fit = curve_fit(ExpFit, x_points, y_points)
        y1_points = fit.(x_points)
        y2_points = log.(y1_points)
        m_mat[j,i] = (y2_points[2]-y2_points[1])/100.0
    end
end
m_matv2 = (10^5).*m_mat

15×23 Array{Float64,2}:
 -617.695   -869.258   -574.653   …  -420.934    -395.393    -387.602  
 -952.827   -798.141   -576.921      -395.006    -389.343    -349.078  
 -584.464   -815.856   -561.067      -412.622    -390.22     -360.691  
 -930.116   -748.076   -713.708      -411.391    -418.233    -343.204  
 -881.554   -810.068   -547.979      -450.482    -404.941    -376.235  
 -693.626   -714.025   -701.961   …  -412.66     -392.589    -449.544  
 -942.257   -821.398   -481.577      -408.599    -323.553    -458.973  
 -311.374   -272.711   -280.707       -47.3723    -38.3207    -36.6116 
 -326.569   -288.472   -251.729       -65.8453    -57.1418    -58.1381 
 -149.006   -117.631   -125.793       -30.3606    -21.4559    -19.8233 
  -61.9742   -51.3691   -48.0338  …    -6.85055    -6.4644     -5.56349
  -33.956    -29.5007   -27.3618       -3.15649    -2.56172    -2.78906
  -24.0926   -20.265    -17.9064       -2.4882     -2.12576    -2.02709
  -21.4351   -15.9605   -14.4241       -

In [38]:
ma = pyimport("numpy.ma")
ticker = pyimport("matplotlib.ticker")
cm = pyimport("matplotlib.cm")

X, Y = np.meshgrid(Re_rangeu,α_rangeu)

Xmin,Xmax = extrema(X)
Ymin,Ymax = extrema(Y)
min,max = extrema(m_matv2)
clev = np.arange(min,max,0.01)
# lev_exp = np.arange(np.floor(np.log10(min)-1),np.ceil(np.log10(max)+1))
# levs = np.power(2, lev_exp)
fig, ax = plt.subplots()
cs = plt.contourf(X, Y, m_matv2, clev, cmap=plt.cm.coolwarm)
cbar = fig.colorbar(cs)
plt.show()

In [39]:
xlabel(L"Re")
ylabel(L"α")

PyObject Text(42.597222222222214, 0.5, '$α$')